In [ ]:
# Imports
import pandas as pd

In [ ]:
# Read in the training data set
df = pd.read_csv('data/train.csv', sep=',',keep_default_na=False, na_values='null')
# Display shape of the data
print(df.shape)
print(df)

In [ ]:
X_train = df.drop(columns=['t2m'])  # Features
y_train = df['t2m']  # Labels (y will be a temperature in K)

# Data Exploration

# Data Wrangling

In [ ]:
def split_times(data):
    data['valid_time'] = pd.to_datetime(data['valid_time'])
    data['hour'] = data['valid_time'].dt.hour
    data['month'] = data['valid_time'].dt.month
    data['day'] = data['valid_time'].dt.day
    return data

In [ ]:
def encode_wind(data):
    return data

In [ ]:
def nearby_data(data, window = 3):
    features = ['u10', 'v10'] # Need to change to be the correct features (dont know what the new wind ones will be called)
    for feature in features:
        data[feature + '_nearby'] = (
            data.groupby(['month', 'day', 'longitude', 'latitude'])[feature]
            .transform(lambda x: x.rolling(window=window, center=True, min_periods=1).mean())
        )
    return data.reset_index()

In [ ]:
def encode_ptype(data):
    return data

In [ ]:
def pca_wind(data):
    return data

In [ ]:
def feature_selection(data):
    return data.drop(columns=['id', 'valid_time', 'day'])

In [ ]:
def data_wrangling_pipeline(data):
    data = split_times(data)
    data = encode_wind(data)
    data = nearby_data(data)
    # TP Square root???
    data = encode_ptype(data)
    data = pca_wind(data)
    data = feature_selection(data)
    return data

data_wrangling_pipeline(X_train)

# Modelling

# Tuning